In [12]:
import pandas as pd

dados = pd.read_csv('reviews.csv')
print(dados.head(10))

                               reviewId app_name  \
0  01e23c15-44bf-46b7-b280-40e880d9d49b   Shopee   
1  df40ba7c-075d-48dc-b798-21466dfb41d6   Shopee   
2  59490b23-b17b-4bba-855b-b5053dadc0db   Shopee   
3  48fd8168-1e42-4475-9a99-935c2fd48769   Shopee   
4  f4e1468b-6d97-4f1a-927d-1afee2bfbe75   Shopee   
5  2ce2db50-eec3-4cf5-afd6-b153ca01cf0f   Shopee   
6  404d8ede-bf3e-4652-a410-ec1ad12e40df   Shopee   
7  0d7c4353-a8f6-467c-abd7-c65b26fd4976   Shopee   
8  5410fc9e-d51f-4f6f-9fdc-01b93eed8750   Shopee   
9  31458c8f-8f11-496f-b72e-dacc9470a52f   Shopee   

                                             content sentiment_polarity  \
0  O app da Shopee é razoável. Apesar de ser fáci...           positive   
1  O app é bem otimizado, fácil de visualizar os ...           positive   
2  O app é muito bom, prático e fácil de entender...           positive   
3  Aplicativo muito bom. Mas algo que me chateou ...           positive   
4  o app é muito bom, porém tenho tido dificuldad...

In [13]:
dados.describe()

,reviewId,app_name,content,sentiment_polarity,sentiment,Unnamed: 5
count,3011,3011,3011,3011,3011,1
unique,3011,10,3011,3,7,1
top,01e23c15-44bf-46b7-b280-40e880d9d49b,Shopee,O app da Shopee é razoável. Apesar de ser fáci...,negative,disgust,X
freq,1,303,1,1742,952,1


In [14]:
#removendo caracteres nao latinos

import regex

dados['content'] = dados['content'].apply(lambda x: regex.sub(r'[^\p{Latin}]', ' ', str(x)))

dados.loc[0, 'content']

'O app da Shopee é razoável  Apesar de ser fácil de utilizar  apresenta alguns bugs  Por exemplo  Sugestão de palavra errada no campo de pesquisa que acarreta na busca errada  Sugestões de produtos que não têm nada a ver com coisas de nosso interesse  Dificuldade de acesso de alguns jogos ou travamento durante alguma jogada  Falhas para carregar certos anexos para avaliação dos produtos  No geral  são problemas que se pode contornar  pois não ocorrem com tanta frequência '

In [15]:
#colocando todo texto para minusculo

dados['content'] = dados['content'].apply(lambda x: str(x).lower())

dados.loc[0, 'content']

'o app da shopee é razoável  apesar de ser fácil de utilizar  apresenta alguns bugs  por exemplo  sugestão de palavra errada no campo de pesquisa que acarreta na busca errada  sugestões de produtos que não têm nada a ver com coisas de nosso interesse  dificuldade de acesso de alguns jogos ou travamento durante alguma jogada  falhas para carregar certos anexos para avaliação dos produtos  no geral  são problemas que se pode contornar  pois não ocorrem com tanta frequência '

In [16]:
import nltk #caso não funcione, adicione a linha nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(texto):
    stops_list = stopwords.words("portuguese")
    word_tokens = word_tokenize(texto)
    texto_sem_stops = [w for w in word_tokens if w not in stops_list]
    return " ".join(texto_sem_stops)
dados['content'] = dados['content'].apply(remove_stopwords)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\biels\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\biels\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
dados.loc[0, 'content']

'app shopee razoável apesar fácil utilizar apresenta alguns bugs exemplo sugestão palavra errada campo pesquisa acarreta busca errada sugestões produtos têm nada ver coisas interesse dificuldade acesso alguns jogos travamento durante alguma jogada falhas carregar certos anexos avaliação produtos geral problemas pode contornar pois ocorrem tanta frequência'

EXTRACAO DAS FEATURES

In [38]:
import pandas as pd
from gensim.models import Word2Vec
from tensorflow.keras import Tokenizer

from tensorflow.keras.preprocessing.text import Tokenizer
import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
import nltk



# Tokenizar as reviews
tokenized_reviews = [simple_tokenize(review) for review in reviews_df['content'].astype(str)]

# Treinar o modelo Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Salvar o modelo para uso posterior
word2vec_model.save('word2vec_reviews.model')

# Preparar os dados para a rede neural
# Usar o Tokenizer para converter as palavras em sequências de inteiros
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews_df['content'].astype(str))
sequences = tokenizer.texts_to_sequences(reviews_df['content'].astype(str))

# Padding das sequências para que todas tenham o mesmo tamanho
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Criar uma matriz de embedding onde cada palavra do vocabulário é substituída pelo vetor do Word2Vec correspondente
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

# Construir o modelo de rede neural
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=100,
                    weights=[embedding_matrix],
                    input_length=max_sequence_length,
                    trainable=False))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

# Para treinar o modelo (exemplo)
# X = padded_sequences
# y = reviews_df['sentiment_polarity'].apply(lambda x: 1 if x == 'positive' else 0)
# model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)


ImportError: cannot import name 'Tokenizer' from 'tensorflow.keras' (C:\Users\biels\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\_tf_keras\keras\__init__.py)

COMO SABEMOS DE ANTEMAO A CATEGORIA CORRETA DO TESTE, VAMOS AVALIAR A PERFORMANCE